This notebook is used to review and edit the context associated with a risk assessment.

1. Set your userkey to access the correct data set, and update the dbhost if necessary.
2. Run the cells to load the current data.
3. Edit the widgets.
4. Press the "Update Scenario" button to save the updated data to the database.  (Do not use the Jupyter Notebook save, that will not keep the data you entered)

In [ ]:
# configuration parameters
userkey = "150"
dbhost  = 'localhost'

In [ ]:

import ipywidgets as widgets
import psycopg2

conn = psycopg2.connect("postgres://csc452:csc452@" + dbhost + "/csc452")  
cursor = conn.cursor()  
cursor.execute('SET mydomain.userkey = ' + userkey)

cursor.execute ('SELECT name, mission, org FROM orgv')
orgs = cursor.fetchall()

cursor.execute('SELECT asset, kind, name, description FROM assetv ORDER BY kind, name ASC')
rows = cursor.fetchall()

cursor.execute('SELECT control, kind, name, description FROM controlv ORDER BY control ASC')
controls = cursor.fetchall()

w = []
w.append (widgets.Text(
    value=orgs[0][0],
    placeholder='organization name',
    description='Name:',
    disabled=False
))

w.append(widgets.Textarea(
    value=orgs[0][1],
    placeholder='organizations mission',
    description='Mission:',
    disabled=False
))

labels=['Proc name:', 'Proc name:', 'System name:', 'System name:', 'System name:', 'Asset name:', 'Asset name:', 'Asset name:', 'Asset name:', 'Asset name:']
wc = []

for x in range(0,10):
    wc.append([])
    wc[x].append(widgets.Text(value=rows[x][2], description=labels[x]))
    wc[x].append(widgets.Textarea(value=rows[x][3], description='Description:'))
    if x > 4:
        y = x - 5
        wc[x].append(widgets.Text(value=controls[y][1], description='Control Type:'))
        wc[x].append(widgets.Textarea(value=controls[y][3], description='Description:'))
        w.append(widgets.HBox([wc[x][0], wc[x][1], wc[x][2], wc[x][3]]))
    else:
        w.append(widgets.HBox([wc[x][0], wc[x][1]]))

button = widgets.Button(description="update scenario")

update_mission = ''' UPDATE orgv SET name=%s, mission=%s WHERE org = %s'''
update_asset = '''UPDATE assetv SET name=%s, description=%s WHERE asset=%s'''
update_control = '''UPDATE controlv SET kind=%s, description=%s WHERE control=%s'''

def on_button_clicked(b):
    cursor.execute (update_mission, (w[0].value, w[1].value, orgs[0][2]))
    for x in range(0,10):
        cursor.execute (update_asset, (wc[x][0].value, wc[x][1].value, rows[x][0]))
        if x > 4:
            y = x - 5
            cursor.execute (update_control, (wc[x][2].value, wc[x][3].value, controls[y][0]))
    conn.commit()

button.on_click(on_button_clicked)

for x in w:
    display(x)
    
display(button)